In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
from mpl_toolkits.mplot3d import Axes3D
from __future__ import print_function, division

In [ ]:
N = 3
def s_to_i(s):
    return sum([2**i * int(b) for i, b in enumerate(s[::-1])])

def bin_str(i):
    """Return a string representation of i with N bits."""
    out = ''
    for j in range(N-1,-1,-1):
        if (i>>j) & 1 == 1:
            out += '1'
        else:
            out += '0'
    return out

def gc(i):
    """Return the Gray code index of i."""
    return i ^ (i >> 1)

def bit_component(x, i):
    """Return i-th bit of x"""
    return (x & 2**i) >> i

def rotate_right(x, d):
    """Rotate x by d bits to the right."""
    d = d % N
    out = x >> d
    for i in range(d):
        bit = (x & 2**i)>>i
        out |= bit << (N+i-d)
    return out

def rotate_left(x, d):
    """Rotate x by d bits to the left."""
    d = d % N
    out = x << d
    excess = out 
    out = out & (2**N-1)
    for i in range(d):
        bit = (x & 2**(N-1-d+1+i))>> (N-1-d+1+i)
        out |= bit << i
    return out

def T(e, d, b):
    """Transform b."""
    out = b ^ e
    return rotate_right(out, d+1)

def T_inv(e, d, b):
    """Inverse transform b."""
    return T(rotate_right(e, d+1), N-d-2, b)


mask = s_to_i('010110')
pi = s_to_i('001000')

In [ ]:
print(mask, pi)

In [ ]:
for i in [15, 12, 8, 11, 16, 19, 23, 20]:
    print(i, gc(i), gcr(i), gcr_inv(gcr(i)))

In [ ]:
def g(i):
    """The direction between subcube i and the next one"""
    return int(np.log2(gc(i)^gc(i+1)))


def d(i):
    """The direction of the arrow whithin a subcube."""
    if i==0:
        return 0
    elif (i%2)==0:
        return g(i-1) % N
    else:
        return g(i) % N

def e(i):
    """Return the entry point of hypercube i."""
    if i==0:
        return 0
    else:
        return gc(2*int(math.floor((i-1)//2)))

def f(i):
    """Return the exit point of hypercube i."""
    return e(2**N-1-i) ^ 2**(N-1)

def i_to_p(i):
    """Extract the 3d position from a 3-bit integer."""
    return [bit_component(i,j) for j in (0,1,2)]

def inverse_gc(g):
    """The inverse gray code."""
    i = g
    j = 1
    while j<N:
        i = i ^ (g >> j)
        j = j + 1
    return i

def gcr(i, mu, pi):
    r = 0
    for k in range(N-1, -1, -1):
        if bit_component(mu, k):
            r = (r << 1) | bit_component(i, k)
    return r

def gcr_inv(r, mu, pi):
    i = 0
    g = 0
    j = sum([bit_component(mu, k) for k in range(N)])-1
    for k in range(N-1, -1, -1):
        if bit_component(mu, k)==1:
            i |= bit_component(r, j) << k
            g |= ( (bit_component(i, k) + bit_component(i, k+1))%2 ) << k
            j -= 1
        else:
            g |= bit_component(pi, k) << k
            i |= ( (bit_component(g, k) + bit_component(i, k+1)) % 2) << k
    return i

M = [3, 4, 4]

def extract_mask(i):
    mu = 0
    for j in range(N-1, -1, -1):
        mu = mu << 1
        if M[j] > i:
            mu = mu | 1
    return mu

def TR_algo7(p):
    h = 0
    ve = 0
    vd = 2
    m = max(M)
    for i in range(m-1, -1, -1):
        mu = extract_mask(i)
        mu_norm = sum([bit_component(mu, j) for j in range(N)])
        mu = rotate_right(mu, vd+1)
        pi = rotate_right(ve, vd+1) & ((~mu) & 2**N-1)
        l = [bit_component(px, i) for px in p]
        # 2. construct a integer whose bits are given by l
        l = sum( [lx*2**j for j, lx in enumerate(l)] )
        l = T(ve, vd, l)
        w = inverse_gc(l)
        r = gcr(w, mu, pi)
        ve = ve ^ rotate_left(e(w), vd+1)
        vd = (vd + d(w) + 1) % N
        h = (h << mu_norm) | r
    return h

def TR_algo8(h):
    ve = 0
    vd = 2
    k = 0
    p = [0,]*N
    m = max(M)
    vM = sum(M)
    for i in range(m-1, -1, -1):
        mu = extract_mask(i)
        mu_norm = sum([bit_component(mu, j) for j in range(N)])
        mu = rotate_right(mu, vd+1)
        pi = rotate_right(ve, vd+1) & (~mu & 2**N-1)
        r = [bit_component(h, vM - k - (j+1)) for j in range(mu_norm)][::-1]
        r = sum( [rx*2**j for j, rx in enumerate(r)] )
        k = k + mu_norm
        w = gcr_inv(r, mu, pi)
        l = gc(w)
        l = T_inv(ve, vd, l)
        for j in range(N):
            p[j] |= bit_component(l, j) << i
        ve = ve ^ (rotate_left(e(w), vd+1))
        vd = (vd + d(w) + 1) % N
    return p

In [ ]:
TR_algo7([2, 1, 0])

In [ ]:
TR_algo8(29)

In [ ]:
d

In [ ]:
print(bin_str(4))
print((~4) & 2**N-1, bin_str(~4))

In [ ]:
bin_str(~4 & 2**N-1)

In [ ]:
import h5py

with h5py.File('/home/pierre/tmp-soft/nano-dimer/test/hilbert_1x3x2x4.h5', 'r') as ref_file:
    ref_dset = ref_file['value']
    ref_data = ref_dset[...]
    ref_M = ref_dset.attrs['M']

In [ ]:
ref_M

In [ ]:
for i in range(10):
    print(i, ref_data[i], TR_algo8(i))

In [ ]:
for i in range(2**N):
    not_i = ~i & 2**N-1
    assert not_i >=0
    assert not_i < 2**N
    assert i & not_i == 0
    assert i | not_i == 2**N-1

In [ ]:
fig = plt.figure(figsize=(14, 12))

ax = fig.add_subplot(111,projection='3d')

test_data = [TR_algo8(i) for i in range(128)]
ax.plot(*zip(*test_data))

for i, (x, y, z) in enumerate(test_data):
    ax.text(x, y, z, str(i))

print(test_data)

plt.savefig('compact_3_2_2.png')

In [ ]:
M = [3, 2, 2]
print(M, 2**sum(M))